In [1]:
# 最小化法による無作為割付

In [2]:
include("../src/Sampling/minimization.jl")
using Printf

In [3]:
# Reference: https://lexjansen.com/nesug/nesug04/ap/ap07.pdf
let
    assigned_treatment = ["B","A","A","B","A","B","A","B","A","A","B","A","A","B","B","B"]
    assigned_patients = [[6,4],[5,3],[5,3],[6,3],[5,4],[5,4],[6,4],[5,3],[6,4],[5,4],[6,3],[5,3],[5,3],[5,3],[5,4],[5,4]]
    assigned_data = collect(zip(assigned_patients, assigned_treatment))
    prob = 1.0
    group_ratio_pairs = [("A",1), ("B",1)]
    
    total_mat = zeros(Int,2,2,2)
    for d in assigned_data
        i = d[2] == "A" ? 1 : 2
        j = d[1][1] == 5 ? 1 : 2
        k = d[1][2] == 3 ? 1 : 2
        total_mat[i,j,k] += 1
    end
    println("Factor | A | B")
    println("(5, 3) | $(total_mat[1,1,1]) | $(total_mat[2,1,1])")
    println("(6, 3) | $(total_mat[1,2,1]) | $(total_mat[2,2,1])")
    println("(5, 4) | $(total_mat[1,1,2]) | $(total_mat[2,1,2])")
    println("(6, 4) | $(total_mat[1,2,2]) | $(total_mat[2,2,2])")
    println("Total  | $(sum(total_mat[1,:,:])) | $(sum(total_mat[2,:,:]))")
    
    new_patient = [5,3]
    treatment = assign(assigned_data, prob, new_patient, group_ratio_pairs, Range())
    println("Assigned new patient $(new_patient) to $treatment .")
end

Factor | A | B
(5, 3) | 4 | 2
(6, 3) | 0 | 2
(5, 4) | 2 | 3
(6, 4) | 2 | 1
Total  | 8 | 8
Assigned new patient [5, 3] to B .


In [4]:
let
    assigned_data = Tuple{Vector{Int},String}[]
    group_ratio_pairs = [("A",1),("B",2),("C",1)]
    prob = 0.9
    total_mat = zeros(Int, 3, 2, 3)
    
    print("Factor | Tr |  ")
    for i in 1:3,j in 1:2, k in 1:3; @printf "%s%d%d, " group_ratio_pairs[i][1] j (k+2); end; print("\n")
    for _ in (1:32)
        f1 = rand() <= 0.7 ? 1 : 2
        f2 = rand() <= 0.5 ? 3 : (rand() <= 0.5 ? 4 : 5)
        treatment = assign(assigned_data, prob, [f1,f2], group_ratio_pairs, Variance())
        push!(assigned_data, ([f1,f2], treatment))
        i = treatment == "A" ? 1 : (treatment == "B" ? 2 : 3)
        j = f1
        k = f2 - 2
        total_mat[i,j,k] += 1
        print("($f1, $f2) |  $treatment | ")
        for _i in 1:3, _j in 1:2, _k in 1:3; @printf "%4d," total_mat[_i,_j,_k]; end; print("\n")
    end
    
    println("=========")
    println("Factor | A | B | C")
    for i in 1:2, j in 1:3
        @printf "(%d, %d) |%2d |%2d |%2d\n" i j+2 total_mat[1,i,j] total_mat[2,i,j] total_mat[3,i,j]
    end
    @printf "Total  |%2d |%2d |%2d\n" sum(total_mat[1,:,:]) sum(total_mat[2,:,:]) sum(total_mat[3,:,:])
end

Factor | Tr |  A13, A14, A15, A23, A24, A25, B13, B14, B15, B23, B24, B25, C13, C14, C15, C23, C24, C25, 
(1, 4) |  B |    0,   0,   0,   0,   0,   0,   0,   1,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
(1, 3) |  C |    0,   0,   0,   0,   0,   0,   0,   1,   0,   0,   0,   0,   1,   0,   0,   0,   0,   0,
(2, 5) |  B |    0,   0,   0,   0,   0,   0,   0,   1,   0,   0,   0,   1,   1,   0,   0,   0,   0,   0,
(1, 4) |  A |    0,   1,   0,   0,   0,   0,   0,   1,   0,   0,   0,   1,   1,   0,   0,   0,   0,   0,
(1, 4) |  B |    0,   1,   0,   0,   0,   0,   0,   2,   0,   0,   0,   1,   1,   0,   0,   0,   0,   0,
(1, 4) |  C |    0,   1,   0,   0,   0,   0,   0,   2,   0,   0,   0,   1,   1,   1,   0,   0,   0,   0,
(1, 4) |  A |    0,   2,   0,   0,   0,   0,   0,   2,   0,   0,   0,   1,   1,   1,   0,   0,   0,   0,
(1, 5) |  B |    0,   2,   0,   0,   0,   0,   0,   2,   1,   0,   0,   1,   1,   1,   0,   0,   0,   0,
(1, 5) |  A |    0,   2,   1,   0,   0,   0,   0,   2,